In [6]:
%run MChain.ipynb

In [7]:
import time

In [8]:
%%time
# birth-death process

N = 100
U = np.eye(N) + np.eye(N,k=-1) + np.eye(N,k=1)
U[N-1,N-2] = 0

mc = MChain()
mc.from_matrix(U)

CPU times: user 88.4 ms, sys: 2.99 ms, total: 91.3 ms
Wall time: 89.3 ms


In [17]:
%%time
mc.analyse()

CPU times: user 16.8 s, sys: 1.91 ms, total: 16.8 s
Wall time: 16.8 s


In [10]:
%%time
trace = mc.run(start='s0',steps=10000)

CPU times: user 188 ms, sys: 43.1 ms, total: 231 ms
Wall time: 184 ms


In [19]:
N = mc.N # number of states 
S = mc.S # state ids
s = mc.s # state names
P = mc.P # transition probability matrix        
R = mc.R # reachability matrix
c = mc.c # state classes
t = mc.t # transient indicator of states
r = mc.r # recurrent indicator of states
p = mc.p # state periodes

In [26]:
%%time
E = -R.astype(int)

#if i0, j0 recurrent, j0 reachable from i0 ==> visits j0 
M0 = np.outer(r,r)
E[M0] = E[M0]<0

#i0 transient, j0 recurrent, j0 reachable from i0, i0 reaches only one class ==> must visits j0
for i0 in S[t]:  #k, i0 in enumerate(S[t]):
    c0 = E[i0][r]*c[r]
    if 1 == len(np.unique(c0[c0<0])):
        E[i0][r] = -E[i0][r]    

P0 = P[t][:,t]
n0 = P0.shape[0]
P0 = np.append(P0,1-np.sum(P0,axis=1,keepdims=True),axis=1)
P0 = np.append(P0,np.zeros((1,n0+1)),axis=0)        
P0[n0,n0] = 1
        
R0 = P0>0
#R1 = mc._getR(R0)
        
s0 = S[t]
n0 = len(s0)

CPU times: user 4.12 ms, sys: 1 ms, total: 5.12 ms
Wall time: 4.49 ms


In [27]:
%%time
for j0 in range(n0):
    R2 = mc._getR(np.delete(np.delete(R0,(j0),axis=0),(j0),axis=1))
    s1 = np.delete(s0,j0)
    for i0 in range(n0-1):
        if not R2[i0,n0-1]:
            E[s1[i0],s0[j0]] = 1

CPU times: user 8.33 s, sys: 975 μs, total: 8.33 s
Wall time: 8.33 s
